Here is a basic tutorial from  https://dmtn-023.lsst.io .  You will notice that some of the commands are different from the site given above; this is due to the fact that some of those commands on the site are obsolete.  

Here we setup the stack using the w_2017_18 tag.
Note: this should all be done within an NCSA terminal session (it will be easiest that way). RUN THIS EVERY TIME.

In [ ]:
%%bash
. /software/lsstsw/stack/loadLSST.bash
setup lsst_distrib -t w_2017_18

Make a folder for the `ci_hsc` data (if the path doesn't exist, make it; anyPath can truly be any path that you want) and then change directories to your new folder. Put git_lfs on your path so that you can use it.  Obviously, change $username to your NCSA username. RUN THIS EVERY TIME.

In [ ]:
%%bash
cd /scratch/$username/anyPath/
export PATH=/ssd/lsstsw/stack/Linux64/git_lfs/2.0.2/bin/:$PATH

You must have LFS up and working so you have to do the following three code cells (you only have to do this once):

In [ ]:
%%bash
git lfs install

Add these 4 lines to the ~/.gitconfig file:
Cache anonymous access to DM Git LFS S3 servers

In [ ]:
[credential "https://lsst-sqre-prod-git-lfs.s3-us-west-2.amazonaws.com"]
  helper = store
[credential "https://s3.lsst.codes"]
  helper = store

Add these 2 lines to ~/.git-credentials (if need be, create one).

In [ ]:
https://:@lsst-sqre-prod-git-lfs.s3-us-west-2.amazonaws.com
https://:@s3.lsst.codes

Here we copy the ci_hsc data from github; you only need to do this once.

In [ ]:
%%bash
git clone https://github.com/lsst/ci_hsc.git

Run ci_hsc automatically;  see README on https://github.com/lsst/ci_hsc if you have questions.  The `scons` command will take a couple hours to go through, so while it is working away, open another terminal and let's try to recreate what it is doing by hand so that we get an idea of the codes being used.

*NOTE*: `scons` does not use `coaddDriver.py` or `multiBandDriver.py` currently (thanks Hsin-Fang!).

In [ ]:
%%bash
cd ci_hsc/
setup -k -r .
scons

If you open a new window to work in while scons is busy, remember to rerun the cd ci_hsc and setup or else this will not work.  Additionally, you should also run the . /software/ and setup lsst command at the top of the page; otherwise this will throw an error. Also, do not forget to cd into `ci_hsc` and run the `setup -k -r .`.  If you don't, you'll throw an error.

Here we are preparing a new data repository in the directory called "DATA2".  Unlike normal files, the files within DATA2 should only be accessed or modified by the butler (lsst.dat.persistence.Butler).  The `echo` command simply tells the butler which mapper our repo is using.  `ingestImages.py` will put the raw image symlinks into our data repo and create a registry of all of the raw images in our repo.

Once we have ingested the raw images, we then create a symlink to our calibration frames, and then we create an additional symlink to our desired reference catalog.  In the dmtn-023 link above, this was done using eups; not exactly sure why we differ in this case from the tutorial. 

*NOTE*: The default reference catalog for HSC was changed from the astrometry_net style to the new HTM style in [DM-9438](https://jira.lsstcorp.org/browse/DM-9438). The default for other cameras are still the astrometry_net style as of today. Supposedly if we tweak the config properly, we can still use the astrometry_net reference catalog for HSC. (Thanks Hsin-Fang!)

In [ ]:
%%bash
mkdir DATA2
echo "lsst.obs.hsc.HscMapper" > DATA2/_mapper
ingestImages.py DATA2 $CI_HSC_DIR/raw/*.fits --mode=link

cd DATA2
ln -s $CI_HSC_DIR/CALIB .
mkdir ref_cats
ln -s $CI_HSC_DIR/ps1_pv3_3pi_20170110 ref_cats/ps1_pv3_3pi_20170110

Instead of doing `singleFrameDriver.py`, we instead do most of the same work with `processCcd.py`.

THREE WAYS OF PROCESSING CCD: 
- One visit and one ccd, 
- one visit with many ccd's, and 
- many visits with many ccd's.

In [ ]:
%%bash
processCcd.py . --rerun example --id visit=903986 ccd=16
processCcd.py . --rerun example --id visit=903334 ccd=16^23
processCcd.py . --rerun example --id visit=903336 ccd=17^24 
    --id visit=903342 ccd=4^10 --id visit=903344 ccd=5^11

When you are making the sky maps, you are essentially making a tract or patch in the sky based on the visit ID's that you specify.  If you want to make a sky map of the whole sky, you simply use the command `makeSkyMap.py`.

(I'm not sure if we got the first command to work, but if you run the next line, you should be fine).

The use of `example:exampleDistSkyMap` allows me to chain together the input and output folders.

In [ ]:
%%bash
makeDiscreteSkyMap.py . --rerun example:exampleDistSkyMap 
    --id visit=903334 ccd=16^23 --id visit=903986 ccd=16 
    --config skyMap.projection="TAN" 
makeSkyMap.py . --rerun example -C $CI_HSC_DIR/skymap.py

These three steps are usually done by the coaddDriver.py when scons is running; this simply builds a coadd patch-by-patch.  Makes lots of different data products (`deepCoadd_tempExp`, `deepCoadd_calexp`, `deepCoadd`, and `deepCoadd_det`) that can be viewed with `butler`.

In [ ]:
%%bash
makeCoaddTempExp.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I -c doApplyUberCal=False --selectId visit=903986 ccd=16 --selectId visit=903334 ccd=16^23 --selectId visit=903336 ccd=17^24 --selectId visit=903342 ccd=4^10 --selectId visit=903344 ccd=5^11
assembleCoadd.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I --selectId visit=903986 ccd=16 --selectId visit=903334 ccd=16^23 --selectId visit=903336 ccd=17^24 --selectId visit=903342 ccd=4^10 --selectId visit=903344 ccd=5^11 --clobber-config (if you do this right the first time, you shouldn't have to clobber)
detectCoaddSources.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I

These four steps are also usually done by multiBandDriver.py in scons:
- First we merge the detections across bands in a patch, which creates the data product `deepCoadd_mergeDet`
- Next, we deblend and measure objects independently in every band, which creates the data product `deepCoadd_meas`
- Then we compare measurements across bands using a reference band for each object to create the data product `deepCoadd_ref`.
- Finally, we measure again in everyband while we take the positions and shapes as constant using the values in the reference band to create `deepCoadd_forced_src` which has flux measurements with the best estimates of colors.

In [ ]:
%%bash
mergeCoaddDetections.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I^HSC-R
measureCoaddSources.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I
mergeCoaddMeasurements.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I^HSC-R
forcedPhotCoadd.py . --rerun example --id patch=5,4 tract=0 filter=HSC-I

Finally, we do forced photometry on exposure images with the coadd reference catalog.  Unfortunately, this doesn't yet have a way to deblend sources.

In [ ]:
%%bash
forcedPhotCcd.py . --rerun example --id visit=903986 ccd=16 tract=0 -C $CI_HSC_DIR/forcedPhotCcdConfig.py

## Using Butler

It is all nice and well that we have made this data above, but without accessing it, it is useless.  So, in order to access it, we will first need to run the first code cell in this notebook. **IF YOU DON'T DO THIS, THE CODE BELOW WILL NOT WORK.**  

Once this is done, if you are in a bash shell you can then start a python inline session by simply entering `python`.  Alternately, if you are in a notebook that is connected to the NCSA server, you can just proceed as shown below.

First you must create a Butler constructor (making sure you are in the DATA directory that you want to analyze:

In [ ]:
from lsst.daf.persistence import Butler
butler = Butler("rerun/example")

Once this is done, you may extract any data products that you've made.  Below are some examples taken directly from https://dmtn-023.lsst.io.  You may change them as you see fit.

*NOTE*: The default of immediate has been changed to true earlier this year (DM-8096).

In [ ]:
calexp = butler.get("calexp", visit=903334, ccd=16, immediate=True)
src = butler.get("src", visit=903334, ccd=16, immediate=True)
skyMap = butler.get("deepCoadd_skyMap", immediate=True)
coadd = butler.get("deepCoadd_calexp", tract=0, patch="1,1", filter="HSC-I", immediate=True)
meas = butler.get("deepCoadd_meas", tract=0, patch="1,1", filter="HSC-I", immediate=True)

Just for a bit of clarity: `immediate` is set to `true` so that Butler will read and return objects immediately.  If immediate is False or simply not set, then you'll have to go through an I/O proxy (really annoying and sometimes confusing). 

Please note that using `butler.get` will not necessarily give output that is human-readable.  Instead, `butler.get` will give output that can be given to other codes.

## ds9: image viewing

In order to do that, you must use ds9.  A good way to do this is to follow the steps taken here: https://community.lsst.org/t/using-an-lsst-dev-stack-w-local-ipython-ds9-tutorial/592

Also, remember: in order to keep your sanity, do this either in a Linux environment (your VM is just fine) or on a Mac; this is NOT easy to do on a Windows machine.

A couple words of warning, however:
- When you create ~/.ssh/config, do not enclose ANYTHING <>. Instead, simply put the desired values in for your username, XXXX, YYYY, and ZZZZ
- REMEMBER TO EXPORT THE XPA PORTS EACH TIME BEFORE YOU LAUNCH DS9
- Please remember to do the long ssh command for the first call of the day; otherwise the localhost call on Firefox will not work
- YOU MUST MANUALLY LAUNCH DS9 EACH TIME BEFORE YOU TRY TO SSH INTO LSST-DEV AND DO NOT EXIT OUT OF DS9 UNTIL YOU ARE DONE WITH YOUR SSH SESSION
- Once you launch ds9, _always_ check in File > XPA > Information to make sure that the second number for XPA_METHOD matches with XXXX that you entered into your export method
- After you ssh into lsst-dev, DO NOT use the method on the website above to setup the stack; instead, do the following:
```
. /software/lsstsw/stack/loadLSST.bash
setup lsst_distrib -t w_2017_18
```
- Do not forget to export the XPA_PORT again!
- Use the first method for launching the Ipython notebook; you shouldn't need a continuous session
- Always try out their test case to make sure everything is working properly: 


```
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import numpy as np

data = np.zeros((10,10), dtype=np.float32)
img = afwImage.ImageF(data)
afwDisplay.mtv(img,frame=0)
```

- Here is another good test case:

```import lsst.afw.display as afwDisplay
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler("/datasets/hsc/repo/rerun/private/hchiang2/RC/DM-10129")
exp = butler.get("raw", visit=350, ccd=3)
afwDisplay.getDisplay().mtv(exp)```

## DMTN Tutorial, Mark-II: From 
## https://dmtn-023.lsst.io/

Now, this tutorial is going to be done for the explicit reason of working on and understanding the maker functions.  This will follow along with DMTN much more closely than above.  Send good vibes people.  And maybe a bottle of Crown and a case of vanilla Coke.

Note: as you are following along, DON'T use the eups command or the setup below it; instead use the ref_cats and ln above.  Below is a rundown of a translation of the DMTN tutorial.  

In [ ]:
%%bash
. /software/lsstsw/stack/loadLSST.bash
setup lsst_distrib -t w_2017_18
cd /scratch/$username/anyPath/
export PATH=/ssd/lsstsw/stack/Linux64/git_lfs/2.0.2/bin/:$PATH
cd ci_hsc/
setup -k -r .

mkdir DATA3
echo "lsst.obs.hsc.HscMapper" > DATA2/_mapper
ingestImages.py DATA2 $CI_HSC_DIR/raw/*.fits --mode=link

cd DATA3
ln -s $CI_HSC_DIR/CALIB .
mkdir ref_cats
ln -s $CI_HSC_DIR/ps1_pv3_3pi_20170110 ref_cats/ps1_pv3_3pi_20170110

Now we do singleFramDriver, which processes the visits.  This can be done one visit at a time, or many at a time. 

In [ ]:
%%bash
singleFrameDriver.py . --rerun example1 --id visit=903334 --cores=4
singleFrameDriver.py . --rerun example1 --cores=4 \
  --id visit=903334..903338:2 --id visit=903342..903346:2 \
  --id visit=903986..903990:2 --id visit=904010^904014

We make the sky maps as above.  Still not sure if makeDiscreteSkyMap is working

In [ ]:
%%bash
makeDiscreteSkyMap.py . --rerun example1:example2 \
  --id visit=903334..903338:2 --id visit=903342..903346:2 \
  --id visit=903986..903990:2 --id visit=904010^904014 \
  --config skyMap.projection="TAN"
makeSkyMap.py . --rerun example1 -C $CI_HSC_DIR/skymap.py

Building the coadds.

In [ ]:
## not sure if these work
coaddDriver.py DATA --rerun example1 \
  --selectId visit=903334..903338:2 --selectId visit=903342..903346:2 \
  --id tract=0 patch=1,1 filter=HSC-R --cores=4 \
  --config assembleCoadd.doApplyUberCal=False \
  makeCoaddTempExp.doApplyUberCal=False
  
coaddDriver.py DATA --rerun example1 \
  --selectId visit=903986..903990:2 --selectId visit=904010^904014 \
  --id tract=0 patch=1,1 filter=HSC-I --cores=4 \
  --config assembleCoadd.doApplyUberCal=False \
  makeCoaddTempExp.doApplyUberCal=False

coaddDriver.py DATA --rerun example1 \
  --selectId visit=903334..903338:2 --selectId visit=903342..903346:2 \
  --id tract=0 patch=0,0^0,1^0,2^1,0^1,2^2,0^2,1^2,2 filter=HSC-R \
  --cores=4
  --config assembleCoadd.doApplyUberCal=False \
  makeCoaddTempExp.doApplyUberCal=False
  
coaddDriver.py DATA --rerun example1 \
  --selectId visit=903986..903990:2 --selectId visit=904010^904014 \
  --id tract=0 patch=0,0^0,1^0,2^1,0^1,2^2,0^2,1^2,2 filter=HSC-I \
  --cores=4
  --config assembleCoadd.doApplyUberCal=False \
  makeCoaddTempExp.doApplyUberCal=False
  

Now, we make the color images.

In [ ]:
multiBandDriver.py . --rerun example1:example3 \
  --id tract=0 patch=1,1 filter=HSC-R^HSC-I \
  --cores=2